## Table of Contents
###  [1.Pre-Processing](pre-processing.ipynb)
###  [2.Data Assimilation](Data Assimilation.ipynb)
###  [3.Post-Processing](post-processing.ipynb)
<div class="toc" style="margin-top: 1em;">
   <ul class="toc-item" id="toc-level0">
      <li><span><a href='#part 3.1' data-toc-modified-id="part 1"><span class="toc-item-num">&nbsp;&nbsp;</span>3.1 Plot estimated permeability over time</a></span></li>
      <li><span><a href='#part 3.2' data-toc-modified-id="part 2"><span class="toc-item-num">&nbsp;&nbsp;</span>3.2 Calculate the vertical hydraulic exchange flux (HEF)</a></span></li>
      <li><span><a href='#part 3.3' data-toc-modified-id="part 3"><span class="toc-item-num">&nbsp;&nbsp;</span>3.3 Compare the calculated HEF with that from temperature method</a></span></li>
</div>





<a id='part 3.1'></a>
# 3.1 Plot estimated permeability over time


In [ ]:
load(paste("results/",tail(system("ls -rt results/",intern=TRUE),1),sep=""))

ntime = length(perm.ls)



perm.ls = as.data.frame(perm.ls)
perm.ls = log10(perm.ls*3600*24*9.81*1000*1000)
colors = rainbow(nreaz)

jpeg(paste("figures/perm.jpg",sep=''),width=8,height=5,units='in',res=300,quality=100)
par(mgp=c(1.8,0.6,0))
plot((1:ntime),perm.ls[1,],type="l",
                                        #     ylim=range(perm.ls),
     ylim = c(-5,5),
     xlab="Time (hr)",
     ylab="Hydraulic conductivity (log10 m/d)"
     )
for (ireaz in 1:nreaz)
{
    lines(1:ntime,perm.ls[ireaz,],type="l",col="lightsalmon3")

}
lines(1:ntime,as.numeric(colMeans(perm.ls)),lwd=2)
## lines(1:length(ref.perm),ref.perm,col="blue",lty=2,lwd=2)

legend("topright",
       c("Mean","Realizations"),
       col=c("black","lightsalmon3"),
       lty=c(1,1),
       bty="n"
       )
dev.off()

<a id='part 3.2'></a>
# 3.2 Calculate the vertical hydraulic exchange flux (HEF)
1. the vertical HEF is calculated by Darcy's law: HEF = hydraculic conductivity * hydraulic gradient


In [ ]:
hef = perm.ls*0.001/(3600*24*9.81*1000)*hg
plot((1:ntime),hef[1,],type="l",
                                        
     ylim = c(-5,5),
     xlab="Time (hr)",
     ylab="HEF (m/d)"
     )

<a id='part 3.3'></a>
# 3.3 Compare the calculated HEF with that from temperature method
1. The daily HEF can also be estimated by temperature data only using VFLUX 2 (Ivrine et al., 2015). The VFLUX 2 is writen in Matlab and will be incorporated into the tutorial later. 
